<a href="https://colab.research.google.com/github/ms337/eegDNNClassifier/blob/master/Loading_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os
import time
import h5py



The data is acquired in two sessions. Each image was presented for 20 to 30 times per session.  Each trial starts from -100 ms to 1200ms from image onset. Image is presented for 500ms.
Stimuli numbers 13 to 24 are human face images (12 conditions), and stimuli 73:76, 78:80, 83:85, 88:89 are small objects (12 conditions).
Use the data in session 1. Please write a script to load data of face images as class1 and data of object images as class2 and then train a SVM classifier to classify these two classes using the data at each time point using a leave one out cross validation procedure and report the classifier accuracy. The classifier is trained and tested on the data at each time point independently.
Then plot the decoding time series (classifier accuracy over time) and write a report describing the method, results and conclusion. Let me know if you have any questions or need more information about the data. 

0 = face
1 = smallObj
Binary Classifier
Each Condition is an Image
Each Image has 20 to 30 trials
Each trial has 1301 time points and 306 parameters



In [0]:
def load_data(classDirectoryPath):
    listOfImgDirs = []
    for root, dirs, files in os.walk(classDirectoryPath, topdown=False):
        for name in dirs:
            listOfImgDirs.append(name)
    listOfImgDirs = sorted(listOfImgDirs)
    
    start=time.time()
    count = 0
    finalMatrix = np.empty((306,1301), order='F')

    for eachImg in listOfImgDirs:
        pathOfCond = os.path.join(classDirectoryPath, eachImg)
#         print("----")
        count = count + 1
#         print(count)
    #     print(pathOfCond)
        for roots, dirs, files in os.walk(pathOfCond):
            for eachTrial in sorted(files):
                if not (eachTrial == ".DS_Store"):
                    loadData = loadmat(os.path.join(pathOfCond,eachTrial))
                    finalMatrix = np.append(finalMatrix,loadData["F"], axis=1)
    #             print(os.path.join(pathOfCond,eachTrial))
    end=time.time()
    print("Time taken: "+ str(end-start))
    finalMatrix = finalMatrix[:,1301:]
    return finalMatrix
    

In [0]:
faces_X = load_data("./face")


Time taken: 44.97403287887573


In [0]:
smallObj_X = load_data("./smallObj")

Time taken: 43.041918992996216


In [0]:
print(faces_X.shape)
faces_X[0][0]

(306, 310939)


-7.532633119546701e-13

In [0]:
np.random.shuffle(faces_X.T)
print(faces_X.shape)

(306, 310939)


In [0]:
np.random.shuffle(smallObj_X.T)
print(smallObj_X.shape)

(306, 313541)


In [0]:
# code to test if shuffling is correct and only columns are switch around
# for j in range(0, 310939):
#     if (faces_X[0][j] == -7.532633119546701e-13):
#         print(j)
#         print("+++")
#         print(faces_X[:, j])


Need to check if by any chance 1301 entries are getting added to the matrix.
Possibly where np.empty is used. 


In [0]:
faces_Y = np.zeros((1, faces_X.shape[1]))
smallObj_Y = np.ones((1, smallObj_X.shape[1]))


In [0]:
combData_X = np.concatenate((faces_X, smallObj_X), axis = 1)
combData_Y = np.concatenate((faces_Y, smallObj_Y), axis = 1)

In [0]:
print(combData_X.shape)
print(combData_Y.shape)

(306, 624480)
(1, 624480)


In [0]:
print(combData_X[124][21418])
print(combData_Y[0][21212])

-5.368013277459939e-12
0.0


In [0]:
permutation = list(np.random.permutation(combData_X.shape[1]))
shuffled_X = combData_X[:, permutation]
shuffled_Y = combData_Y[:, permutation]
print(shuffled_X.shape)
print(shuffled_Y.shape)

(306, 624480)
(1, 624480)


In [0]:
print(shuffled_X[124][21418])
print(shuffled_Y[0][21212])


3.2787368105211703e-12
1.0


In [0]:
# #Check if corresponding columns in X and Y are labelled correctly
# for j in range(0, 624480):
#     if (shuffled_X[0][j] == -1.5532603192192899e-13):
#         print(j)
#         print("+++")
#         print(shuffled_X[:, j])
#         print(shuffled_Y[0][j])

In [0]:
sizeOfTrain = int(0.9*shuffled_X.shape[1])
sizeOfDevTest = int(0.95*shuffled_X.shape[1])
print(sizeOfTrain)
print(sizeOfDevTest)
X_train = shuffled_X[:, :sizeOfTrain]
Y_train = shuffled_Y[:, :sizeOfTrain]

X_dev = shuffled_X[:, sizeOfTrain:sizeOfDevTest]
Y_dev = shuffled_Y[:, sizeOfTrain:sizeOfDevTest]

X_test = shuffled_X[:, sizeOfDevTest:]
Y_test = shuffled_Y[:, sizeOfDevTest:]


562032
593256


In [0]:
print(X_train.shape)
print(Y_train.shape)

print(X_dev.shape)
print(Y_dev.shape)

print(X_test.shape)
print(Y_test.shape)

(306, 562032)
(1, 562032)
(306, 31224)
(1, 31224)
(306, 31224)
(1, 31224)


In [0]:
dFile = h5py.File("./data.h5", mode = 'w')
dFile.create_dataset("X_train", data=X_train)
dFile.create_dataset("Y_train", data=Y_train)
dFile.create_dataset("X_dev", data=X_dev)
dFile.create_dataset("Y_dev", data=Y_dev)
dFile.create_dataset("X_test", data=X_test)
dFile.create_dataset("Y_test", data=Y_test)
dFile.close()